### Carregando os pacotes necessários

In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
!pip install keras
!pip install tensorflow

^C
^C


In [2]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from keras import optimizers
from keras.models import Model
import pickle

In [3]:
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

### Criando as variáveis

In [4]:
path = os.path.abspath(os.path.join('..', 'input'))
path_data_matches = os.path.join(path, 'matches.csv')

In [5]:
seed = 7
np.random.seed(seed)

In [6]:
dataframe = pd.read_csv(path_data_matches)

In [7]:
dataframe

,year,match_number,team_a,goals_team_a,goals_team_b,team_b,score_diff,winner,matches_played_a,matches_won_a,...,podium_score_cum_a,matches_won_cum_percent_a,matches_played_b,matches_won_b,podium_score_b,cups_won_b,matches_played_cum_b,matches_won_cum_b,podium_score_cum_b,matches_won_cum_percent_b
0,1991,1,China PR,4,0,Norway,4,a,3,2,...,0,66.666667,6,4,8,0,6,4,8,66.666667
1,1991,23,Sweden,1,4,Norway,-3,b,6,4,...,4,66.666667,6,4,8,0,6,4,8,66.666667
2,1991,13,China PR,4,1,New Zealand,3,a,3,2,...,0,66.666667,3,0,0,0,3,0,0,0.000000
3,1991,2,Denmark,3,0,New Zealand,3,a,3,1,...,0,33.333333,3,0,0,0,3,0,0,0.000000
4,1991,8,Norway,4,0,New Zealand,4,a,6,4,...,8,66.666667,3,0,0,0,3,0,0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,2019,27,South Africa,0,4,Germany,-4,b,3,0,...,0,0.000000,5,4,0,2,39,30,44,76.923077
241,2019,29,Jamaica,1,4,Australia,-3,b,3,0,...,0,0.000000,3,2,0,0,20,7,0,35.000000
242,2019,33,Netherlands,2,1,Canada,1,a,7,6,...,8,70.000000,4,2,0,0,22,8,2,36.363636
243,2019,42,Sweden,1,0,Canada,1,a,7,5,...,20,65.714286,4,2,0,0,22,8,2,36.363636


In [8]:
dataframe_dp = dataframe[["year","matches_won_cum_percent_a","podium_score_cum_a","matches_won_cum_percent_b","podium_score_cum_b","winner"]]

In [9]:
dataframe_dp

,year,matches_won_cum_percent_a,podium_score_cum_a,matches_won_cum_percent_b,podium_score_cum_b,winner
0,1991,66.666667,0,66.666667,8,a
1,1991,66.666667,4,66.666667,8,b
2,1991,66.666667,0,0.000000,0,a
3,1991,33.333333,0,0.000000,0,a
4,1991,66.666667,8,0.000000,0,a
...,...,...,...,...,...,...
240,2019,0.000000,0,76.923077,44,b
241,2019,0.000000,0,35.000000,0,b
242,2019,70.000000,8,36.363636,2,a
243,2019,65.714286,20,36.363636,2,a


In [10]:
dataset = dataframe_dp.values

In [11]:
dataset

array([[1991, 66.66666666666666, 0, 66.66666666666666, 8, 'a'],
       [1991, 66.66666666666666, 4, 66.66666666666666, 8, 'b'],
       [1991, 66.66666666666666, 0, 0.0, 0, 'a'],
       ...,
       [2019, 70.0, 8, 36.36363636363637, 2, 'a'],
       [2019, 65.71428571428571, 20, 36.36363636363637, 2, 'a'],
       [2019, 70.0, 8, 48.27586206896552, 24, 'a']], dtype=object)

In [12]:
X = dataset[:,0:5].astype(np.float32)
Y = dataset[:,5]

In [13]:
X

array([[1.9910000e+03, 6.6666664e+01, 0.0000000e+00, 6.6666664e+01,
        8.0000000e+00],
       [1.9910000e+03, 6.6666664e+01, 4.0000000e+00, 6.6666664e+01,
        8.0000000e+00],
       [1.9910000e+03, 6.6666664e+01, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00],
       ...,
       [2.0190000e+03, 7.0000000e+01, 8.0000000e+00, 3.6363636e+01,
        2.0000000e+00],
       [2.0190000e+03, 6.5714287e+01, 2.0000000e+01, 3.6363636e+01,
        2.0000000e+00],
       [2.0190000e+03, 7.0000000e+01, 8.0000000e+00, 4.8275864e+01,
        2.4000000e+01]], dtype=float32)

In [14]:
encoder = LabelEncoder()
encoder.fit(Y)
Y = encoder.transform(Y)

In [15]:
Y

array([0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0,
       0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0,
       0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0])

### Criando o modelo

In [16]:
def create_baseline():
    # Criar Modelo
    model = Sequential()
    model.add(Dense(10, input_dim=5, kernel_initializer='normal', activation='relu'))     
    model.add(Dense(8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(6, kernel_initializer='normal', activation='relu'))
    model.add(Dense(4, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid')) 
    # Compilar Modelo
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 
    return model
    model.save('final_model.pkl')

In [17]:
create_baseline()

In [18]:
estimator = KerasClassifier(build_fn=create_baseline, nb_epoch=100, batch_size=5, verbose=1)

<ipython-input-18-9b18f3aa7767>:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  estimator = KerasClassifier(build_fn=create_baseline, nb_epoch=100, batch_size=5, verbose=1)


In [20]:
estimator

In [21]:
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, Y, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

5/5 [==============================] - 0s 3ms/step - loss: 0.6701 - accuracy: 0.6250
Results: 62.45% (1.39%)


In [22]:
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, Y, test_size=0.20)

In [23]:
estimator.fit(X_train, Y_train)

40/40 [==============================] - 1s 3ms/step - loss: 0.6813 - accuracy: 0.6173


### Olhando as previsões

In [24]:
predictions = estimator.predict(X_test)
print(predictions)

2/2 [==============================] - 0s 3ms/step
[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]]


In [25]:
print(accuracy_score(Y_test, predictions))

0.6530612244897959


In [26]:
encoder = LabelEncoder()
encoder.fit(Y)
Y = encoder.transform(Y)

In [27]:
Y

array([0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0,
       0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0,
       0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0], dtype=int64)

In [28]:
predictions_encoded = encoder.inverse_transform(predictions)
print(predictions_encoded)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0]


C:\Users\User\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:154: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [29]:
probs = estimator.predict_proba(X_test)
print(probs)

2/2 [==============================] - 0s 3ms/step
[[0.56802297 0.431977  ]
 [0.5690671  0.43093288]
 [0.5685619  0.4314381 ]
 [0.5687349  0.4312651 ]
 [0.56816745 0.43183252]
 [0.5681673  0.4318327 ]
 [0.5691018  0.4308982 ]
 [0.5681231  0.43187693]
 [0.5693954  0.4306046 ]
 [0.56913435 0.43086568]
 [0.569384   0.43061605]
 [0.56869197 0.431308  ]
 [0.56754637 0.43245366]
 [0.56939685 0.43060318]
 [0.56942195 0.43057805]
 [0.56930643 0.43069357]
 [0.56902725 0.43097275]
 [0.5687283  0.4312717 ]
 [0.56880116 0.43119887]
 [0.56864506 0.43135494]
 [0.5677259  0.43227413]
 [0.56874835 0.43125162]
 [0.5683284  0.4316716 ]
 [0.5676465  0.4323535 ]
 [0.5684846  0.4315154 ]
 [0.56843007 0.43156993]
 [0.56836116 0.43163887]
 [0.5680606  0.43193942]
 [0.5682371  0.43176287]
 [0.56932205 0.43067795]
 [0.5695512  0.43044874]
 [0.56880116 0.43119887]
 [0.56937885 0.43062115]
 [0.5676334  0.43236658]
 [0.5676902  0.4323098 ]
 [0.56803477 0.4319652 ]
 [0.5688     0.43120003]
 [0.5683576  0.4316424 ]